# Introduction à MongoDB - *évaluation*


## Restaurants 

In [1]:
import pymongo
import pandas

client = pymongo.MongoClient()
db = client.test

### Lister tous les restaurants de la chaîne "Bareburger" (rue, quartier)

In [2]:
pandas.DataFrame(db.restaurants.find(
    { "name": "Bareburger" },
    { "_id": 0, "borough": 1, "address.street": 1 }
))

,address,borough
0,{'street': '31 Avenue'},Queens
1,{'street': 'Laguardia Place'},Manhattan
2,{'street': '7 Avenue'},Brooklyn
3,{'street': '31 Street'},Queens
4,{'street': '8 Avenue'},Manhattan
5,{'street': '2 Avenue'},Manhattan
6,{'street': 'Austin Street'},Queens
7,{'street': 'Bell Boulevard'},Queens
8,{'street': 'Court St'},Brooklyn
9,{'street': 'West 46 Street'},Manhattan


### Lister les trois chaînes de restaurant les plus présentes

In [3]:
pandas.DataFrame(db.restaurants.aggregate([
    { "$sortByCount": "$name" },
    { "$limit": 3 }
]))

,_id,count
0,Subway,421
1,Starbucks Coffee,223
2,Mcdonald'S,208


### Donner les 10 styles de cuisine les plus présents dans la collection

In [4]:
pandas.DataFrame(db.restaurants.aggregate([
    { "$sortByCount": "$cuisine" },
    { "$limit": 10 }
]))

,_id,count
0,American,6183
1,Chinese,2418
2,Café/Coffee/Tea,1214
3,Pizza,1163
4,Italian,1069
5,Other,1011
6,"Latin (Cuban, Dominican, Puerto Rican, South &...",850
7,Japanese,760
8,Mexican,754
9,Bakery,691


### Lister les 10 restaurants les moins bien notés (note moyenne la plus haute)

In [5]:
pandas.DataFrame(db.restaurants.aggregate([
    { "$unwind": "$grades" },
    { "$group": {
        "_id": { "name": "$name", "id": "restaurant_id" },
        "score": { "$avg": "$grades.score" }
    }},
    { "$sort": { "score": -1 }},
    { "$limit": 10 }
]))

,_id,score
0,"{'name': 'Juice It Health Bar', 'id': 'restaur...",75.0
1,"{'name': 'Golden Dragon Cuisine', 'id': 'resta...",73.0
2,"{'name': 'Palombo Pastry Shop', 'id': 'restaur...",69.0
3,"{'name': 'Chelsea'S Juice Factory', 'id': 'res...",69.0
4,"{'name': 'Koyla', 'id': 'restaurant_id'}",61.0
5,"{'name': 'K & D Internet Inc', 'id': 'restaura...",61.0
6,"{'name': 'Ivory D O S Inc', 'id': 'restaurant...",60.0
7,"{'name': 'Ab Halal Restaurant', 'id': 'restaur...",58.0
8,"{'name': 'Espace', 'id': 'restaurant_id'}",56.0
9,"{'name': 'Kikka At Whole Foods Chelsea', 'id':...",56.0


### Lister par quartier le nombre de restaurants, le score moyen et le pourcentage moyen d'évaluation A

In [6]:
pandas.DataFrame(db.restaurants.aggregate([
    { "$addFields": { "eval": { "$first": "$grades" }}},
    { "$group": {
        "_id": "$borough",
        "NbRestaurants": { "$sum": 1 },
        "ScoreMoyen": { "$avg": "$eval.score" },
        "NbA": { "$sum": { "$cond": { "if": { "$eq": [ "$eval.grade", "A"]}, "then": 1, "else": 0 }}}
    }},
    { "$addFields": {
        "PctA": { "$multiply": [ { "$divide": [ "$NbA", "$NbRestaurants" ]}, 100]}
    }}
]))

,_id,NbRestaurants,ScoreMoyen,NbA,PctA
0,Missing,51,8.520000,50,98.039216
1,Brooklyn,6086,10.723014,4957,81.449228
2,Staten Island,969,10.773262,785,81.011352
3,Manhattan,10259,10.952729,8450,82.366702
4,Bronx,2338,10.672095,1926,82.378101
5,Queens,5656,11.075630,4519,79.897454


### Questions complémentaires

Nécessitent une recherche sur la toile pour compléter ce qu’on a déjà vu dans ce TP.

### Lister les restaurants (nom et rue uniquement) situés sur une rue ayant le terme “Union” dans le nom

In [7]:
pandas.DataFrame(db.restaurants.find(
    { "name": {"$regex": "union", "$options": "i" }},
    { "_id": 0, "name": 1, "address.street": 1 }
))

,address,name
0,{'street': 'Henderson Avenue'},Union Hall
1,{'street': 'East 69 Street'},Union Club Of City Of New York
2,{'street': 'East 37 Street'},The Union League Club
3,{'street': 'Union Street'},Union Cafe Restaurant
4,{'street': 'Union Avenue'},Union Pool
5,{'street': 'East 17 Street'},Union Square Theater
6,{'street': '6 Avenue'},Union Bank Of Switzerland
7,{'street': 'Union Street'},Union Hall
8,{'street': '44 Road'},U.N. Federal Credit Union
9,{'street': 'Corona Avenue'},La Union Restaurant


### Lister les restaurants ayant eu une visite le 1er février 2014

In [8]:
from datetime import datetime

pandas.DataFrame(db.restaurants.find(
    { "grades.date": datetime(2014, 2, 1) }
))

,_id,address,borough,cuisine,grades,name,restaurant_id
0,60006d6aa7aafd5a6d45ce69,"{'building': '300', 'coord': [-73.8787725, 40....",Bronx,Hamburgers,"[{'date': 2014-02-01 00:00:00, 'grade': 'A', '...",Mcdonald'S,40395653
1,60006d6aa7aafd5a6d45d0ec,"{'building': '15705', 'coord': [-73.8400791000...",Queens,"Juice, Smoothies, Fruit Salads","[{'date': 2015-01-15 00:00:00, 'grade': 'A', '...",Gold'S Gym,40534973
2,60006d6ba7aafd5a6d45dac1,"{'building': '706', 'coord': [-73.8668038, 40....",Bronx,Chinese,"[{'date': 2014-08-11 00:00:00, 'grade': 'A', '...",K & Q Restaruant,40947971
3,60006d6ba7aafd5a6d45dc0d,"{'building': '427', 'coord': [-73.9740765, 40....",Brooklyn,Pizza,"[{'date': 2014-07-10 00:00:00, 'grade': 'A', '...",Knapp Pizza Iii,40993361
4,60006d6ba7aafd5a6d45e1e3,"{'building': '9718', 'coord': [-73.88871520000...",Brooklyn,Chinese,"[{'date': 2014-02-01 00:00:00, 'grade': 'A', '...",Win Hing Chinese Restaurant,41152860
5,60006d6ba7aafd5a6d45e792,"{'building': '14908', 'coord': [-73.8153339, 4...",Queens,Korean,"[{'date': 2014-07-23 00:00:00, 'grade': 'B', '...",Chinese House Restaurant,41264537
6,60006d6ba7aafd5a6d45e7a0,"{'building': '2087', 'coord': [-73.9615235, 40...",Brooklyn,Chinese/Japanese,"[{'date': 2014-07-30 00:00:00, 'grade': 'A', '...",Sake,41265584
7,60006d6ca7aafd5a6d45ed39,"{'building': '133-45', 'coord': [-73.832814, 4...",Queens,American,"[{'date': 2014-08-19 00:00:00, 'grade': 'A', '...",Roosevelt Sports Bar,41361603
8,60006d6ca7aafd5a6d45f537,"{'building': '1050', 'coord': [-73.86184589999...",Brooklyn,American,"[{'date': 2014-02-01 00:00:00, 'grade': 'A', '...",Forbell Cafe (U.S. Post Office Cafeteria),41460812
9,60006d6ca7aafd5a6d45f66d,"{'building': '300', 'coord': [-73.989723, 40.7...",Manhattan,Hamburgers,"[{'date': 2014-02-01 00:00:00, 'grade': 'A', '...",Shake Shack,41474503


### Lister les restaurants situés entre les longitudes -74.2 et -74.1 et les lattitudes 40.5 et 40.6

In [9]:
pandas.DataFrame(db.restaurants.find(
    { 
        "address.coord.0": { "$gte": -74.2, "$lte": -74.1 },
        "address.coord.1": { "$gte":  40.1, "$lte":  40.6 }
    }
))

,_id,address,borough,cuisine,grades,name,restaurant_id
0,60006d6aa7aafd5a6d45cace,"{'building': '73', 'coord': [-74.1178949, 40.5...",Staten Island,Delicatessen,"[{'date': 2014-11-18 00:00:00, 'grade': 'A', '...",Plaza Bagels & Deli,40364286
1,60006d6aa7aafd5a6d45cad0,"{'building': '203', 'coord': [-74.152359199999...",Staten Island,Delicatessen,"[{'date': 2015-01-05 00:00:00, 'grade': 'A', '...",B & M Hot Bagel & Grocery,40364299
2,60006d6aa7aafd5a6d45cae5,"{'building': '37', 'coord': [-74.138263, 40.54...",Staten Island,American,"[{'date': 2014-04-22 00:00:00, 'grade': 'A', '...",Great Kills Yacht Club,40364610
3,60006d6aa7aafd5a6d45cb11,"{'building': '3823', 'coord': [-74.16536339999...",Staten Island,American,"[{'date': 2014-07-15 00:00:00, 'grade': 'B', '...",Joyce'S Tavern,40365692
4,60006d6aa7aafd5a6d45cb43,"{'building': '135', 'coord': [-74.104655999999...",Staten Island,American,"[{'date': 2014-07-10 00:00:00, 'grade': 'A', '...",Richmond County Country Club,40366928
...,...,...,...,...,...,...,...
327,60006d6ea7aafd5a6d462cf0,"{'building': '14', 'coord': [-74.150459, 40.55...",Staten Island,Other,[],Famous Falafel,50018576
328,60006d6ea7aafd5a6d462d14,"{'building': '2224', 'coord': [-74.1033838, 40...",Staten Island,Other,[],Ciminna Cafe,50018635
329,60006d6ea7aafd5a6d462d58,"{'building': '365', 'coord': [-74.1102822, 40....",Staten Island,Other,[],,50018782
330,60006d6ea7aafd5a6d462d7f,"{'building': '3237', 'coord': [-74.16969230000...",Staten Island,Other,[],Jimmy John'S,50018887


## AirBnB

Nous allons travailler sur des données AirBnB. Celles-ci sont stockées sur le serveur Mongo dans la collection `listingsAndReviews` de la base `sample_airbnb`.

> [Aide sur les données](https://docs.atlas.mongodb.com/sample-data/sample-airbnb)

Une fois créée la connexion à la collection dans Python, répondre aux questions suivantes :

In [10]:
db = client.sample_airbnb

### Lister les différentes types de logements possibles cf (`room_type`)

In [11]:
pandas.DataFrame(db.listingsAndReviews.distinct("room_type"))

,0
0,Entire home/apt
1,Private room
2,Shared room


### Lister les différents équipements possibles cf (`amenities`)

In [12]:
pandas.DataFrame(db.listingsAndReviews.distinct("amenities"))

,0
0,
1,24-hour check-in
2,Accessible-height bed
3,Accessible-height toilet
4,Air conditioning
...,...
181,Wifi
182,Window guards
183,toilet
184,translation missing: en.hosting_amenity_49


### Donner le nombre de logements

In [13]:
db.listingsAndReviews.count_documents({})

5555

### Donner le nombre de logements de type "Entire home/apt"

In [14]:
db.listingsAndReviews.count_documents({ "room_type": "Entire home/apt" })

3489

### Donner le nombre de logements proposant la "TV" et le "Wifi (cf `amenities`) 

In [15]:
db.listingsAndReviews.count_documents({ "amenities": { "$in": [ "TV", "Wifi" ]} })

5443

### Donner le nombre de logements n'ayant eu aucun avis

In [16]:
db.listingsAndReviews.count_documents({ "reviews": [] })

1632

In [17]:
db.listingsAndReviews.count_documents({ "reviews": { "$size": 0 } })

1632

### Lister les informations du logement "10545725" (cf _id)

In [18]:
pandas.DataFrame(db.listingsAndReviews.find({ "_id": "10545725" }))

,_id,listing_url,name,summary,space,description,neighborhood_overview,notes,transit,access,...,monthly_price,cleaning_fee,extra_people,guests_included,images,host,address,availability,review_scores,reviews
0,10545725,https://www.airbnb.com/rooms/10545725,Cozy bedroom Sagrada Familia,Cozy bedroom next to the church Sagrada Famíli...,Cozy beroom located three minutes from the Sag...,Cozy bedroom next to the church Sagrada Famíli...,Well located on a calm residential area a few ...,,Metro on the corner 5 stops away from the Ramb...,,...,1080.0,20.0,0.0,1,"{'thumbnail_url': '', 'medium_url': '', 'pictu...","{'host_id': '1929411', 'host_url': 'https://ww...","{'street': 'Barcelona, Catalunya, Spain', 'sub...","{'availability_30': 0, 'availability_60': 0, '...","{'review_scores_accuracy': 10, 'review_scores_...","[{'_id': '62460002', 'date': 2016-02-14 05:00:..."


### Lister le nom, la rue et le pays des logements dont le prix est supérieur à 10000

In [19]:
pandas.DataFrame(db.listingsAndReviews.find(
    { "price": { "$gte": 10000 }},
    { "_id": 0, "name": 1, "address.street": 1, "address.country": 1}
))

,name,address
0,Apartamento de luxo em Copacabana - 4 quartos,"{'street': 'Rio de Janeiro, Rio de Janeiro, Br..."
1,良德街3号温馨住宅,"{'street': 'HK, Hong Kong', 'country': 'Hong K..."
2,İstanbul un kalbi sisli. Center of istanbul sisli,"{'street': 'Şişli, İstanbul, Turkey', 'country..."
3,`LM 三個睡房的整间公寓,"{'street': 'Hong Kong, Kowloon, Hong Kong', 'c..."
4,HS1-2人大床房+丰泽､苏宁､百脑汇+女人街+美食中心,"{'street': 'Hong Kong, Kowloon, Hong Kong', 'c..."
5,5 PEOPLE ROOM ( 1 TRIP and 1 DOUBLE),"{'street': 'Hong Kong, Kowloon, Hong Kong', 'c..."


### Donner le nombre de logements par type

In [20]:
pandas.DataFrame(db.listingsAndReviews.aggregate([
    { "$sortByCount": "$room_type" }
]))

,_id,count
0,Entire home/apt,3489
1,Private room,1983
2,Shared room,83


### Donner le nombre de logements par pays

In [21]:
pandas.DataFrame(db.listingsAndReviews.aggregate([
    { "$sortByCount": "$address.country" }
]))

,_id,count
0,United States,1222
1,Turkey,661
2,Canada,649
3,Spain,633
4,Australia,610
5,Brazil,606
6,Hong Kong,600
7,Portugal,555
8,China,19


### On veut représenter graphiquement la distribution des prix, il nous faut donc récupérer uniquement les tarifs 
    
Un tarif apparraissant plusieurs fois dans la base doit être présent plusieurs fois dans cette liste

In [22]:
pandas.DataFrame(db.listingsAndReviews.find({}, { "_id": 0, "price": 1 }))

,price
0,80.0
1,317.0
2,115.0
3,40.0
4,701.0
...,...
5550,58.0
5551,501.0
5552,101.0
5553,227.0


### Calculer pour chaque type de logements (`room_type`) le prix (`price`)

In [23]:
pandas.DataFrame(db.listingsAndReviews.aggregate([
    { "$group": { "_id": "$room_type", "price": { "$avg": "$price" }}},
    { "$sort": { "price": -1 }},
    { "$project": { "_id": 0, "type": "$_id", "price": { "$round": [ "$price", 2 ]}}}
]))

,type,price
0,Shared room,349.59
1,Entire home/apt,314.86
2,Private room,212.30


### On veut représenter la distribution du nombre d'avis. Il faut donc calculer pour chaque logement le nombre d'avis qu'il a eu (cf `reviews`)

In [24]:
pandas.DataFrame(db.listingsAndReviews.aggregate([
    { "$project": { "nb_reviews": { "$size": "$reviews" }}}
]))

,_id,nb_reviews
0,10006546,51
1,10009999,0
2,1001265,96
3,10021707,1
4,10030955,0
...,...,...
5550,9983221,107
5551,9985696,0
5552,9987200,90
5553,9990304,13


### Compter le nombre de logement pour chaque équipement possible

In [25]:
pandas.DataFrame(db.listingsAndReviews.aggregate([
    { "$unwind": "$amenities" },
    { "$sortByCount": "$amenities" }
]))

,_id,count
0,Wifi,5303
1,Essentials,5048
2,Kitchen,4951
3,TV,4295
4,Hangers,4226
...,...,...
181,Kayak,1
182,Snorkeling equipment,1
183,Toaster,1
184,Swimming pool,1


### On souhaite connaître les 10 utilisateurs ayant fait le plus de commentaires

In [26]:
pandas.DataFrame(db.listingsAndReviews.aggregate([
    { "$unwind": "$reviews" },
    { "$group": {
        "_id": {"nom": "$reviews.reviewer_name", "id": "$reviews.reviewer_id"},
        "nb_reviews": { "$sum": 1 }
    }},
    { "$sort": { "nb_reviews": -1 }},
    { "$limit": 10 }
]))

,_id,nb_reviews
0,"{'nom': 'Filipe', 'id': '20775242'}",24
1,"{'nom': 'Nick', 'id': '67084875'}",13
2,"{'nom': 'Uge', 'id': '2961855'}",10
3,"{'nom': 'Lisa', 'id': '20991911'}",9
4,"{'nom': 'Thien', 'id': '162027327'}",9
5,"{'nom': 'Lisa', 'id': '69140895'}",8
6,"{'nom': 'Todd', 'id': '60816198'}",8
7,"{'nom': 'Jodi', 'id': '12679057'}",8
8,"{'nom': 'David', 'id': '1705870'}",8
9,"{'nom': 'Courtney', 'id': '55241576'}",8
